# Fake face generator
Use StyleGAN pre-tained model to generate fake face.

[Github address](https://github.com/NVlabs/stylegan)

## Step 1: Install the environment

### System requirements

* Both Linux and Windows are supported, but we strongly recommend Linux for performance and compatibility reasons.

* 64-bit Python 3.6 installation. We recommend Anaconda3 with numpy 1.14.3 or newer.

* TensorFlow 1.10.0 or newer with GPU support.

* One or more high-end NVIDIA GPUs with at least 11GB of DRAM. We recommend NVIDIA DGX-1 with 8 Tesla V100 GPUs.

* NVIDIA driver 391.35 or newer, CUDA toolkit 9.0 or newer, cuDNN 7.3.1 or newer.


In [ ]:
# make sure GPU and CUDA is working
! nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [ ]:
# change tensorflow version
# Attension: CANNOT pip install tensorflow=1.xx.x. It will bring new errors.
# Referece: https://colab.research.google.com/notebooks/tensorflow_version.ipynb
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
# Install the dnnlib
!pip install https://github.com/podgorskiy/dnnlib/releases/download/0.0.1/dnnlib-0.0.1-py3-none-any.whl

## Step 2: Mount the drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.chdir('/content/gdrive/MyDrive/ISY5002PracticeModule/colab')

## Step 3: Make sure GPU available

In [ ]:
# To check if the GPU available.
# If not, make sure you open the GPU configuration.
import tensorflow as tf
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
# Same as above.
tf.test.is_gpu_available()

True

## Step 4: Now run the code!

### Demo from github

In [ ]:
# Copyright (c) 2019, NVIDIA CORPORATION. All rights reserved.
#
# This work is licensed under the Creative Commons Attribution-NonCommercial
# 4.0 International License. To view a copy of this license, visit
# http://creativecommons.org/licenses/by-nc/4.0/ or send a letter to
# Creative Commons, PO Box 1866, Mountain View, CA 94042, USA.

"""Minimal script for generating an image using pre-trained StyleGAN generator."""

import os
import pickle
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib

result_dir = 'results'
cache_dir = 'cache'
run_dir_ignore = ['results', 'datasets', 'cache']

def main():
    # Initialize TensorFlow.
    tflib.init_tf()

    # Load pre-trained network.
    url = './models/karras2019stylegan-ffhq-1024x1024.pkl' # karras2019stylegan-ffhq-1024x1024.pkl
    # with dnnlib.util.open_url(url, cache_dir=cache_dir) as f:
        # _G, _D, Gs = pickle.load(f)
        # _G = Instantaneous snapshot of the generator. Mainly useful for resuming a previous training run.
        # _D = Instantaneous snapshot of the discriminator. Mainly useful for resuming a previous training run.
        # Gs = Long-term average of the generator. Yields higher-quality results than the instantaneous snapshot.
    
    _G, _D, Gs = pickle.load(open(url, 'rb'))

    # Print network details.
    Gs.print_layers()

    # Pick latent vector.
    ## change here to get a different image. The randomState function make sure the same result with a same seed.
    ## https://numpy.org/doc/1.16/reference/generated/numpy.random.RandomState.html
    rnd = np.random.RandomState(3) # 5 
    latents = rnd.randn(1, Gs.input_shape[1])

    # Generate image.
    fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    images = Gs.run(latents, None, truncation_psi=0.7, randomize_noise=True, output_transform=fmt)

    # Save image.
    os.makedirs(result_dir, exist_ok=True)
    png_filename = os.path.join(result_dir, 'example2.png')
    PIL.Image.fromarray(images[0], 'RGB').save(png_filename)

if __name__ == "__main__":
    main()


Gs                              Params    OutputShape          WeightShape     
---                             ---       ---                  ---             
latents_in                      -         (?, 512)             -               
labels_in                       -         (?, 0)               -               
lod                             -         ()                   -               
dlatent_avg                     -         (512,)               -               
G_mapping/latents_in            -         (?, 512)             -               
G_mapping/labels_in             -         (?, 0)               -               
G_mapping/PixelNorm             -         (?, 512)             -               
G_mapping/Dense0                262656    (?, 512)             (512, 512)      
G_mapping/Dense1                262656    (?, 512)             (512, 512)      
G_mapping/Dense2                262656    (?, 512)             (512, 512)      
G_mapping/Dense3                262656 

### generate multiplate images

修改下面的IMAGE_NUMBER变量，决定生成多少张图片

In [ ]:
import os
import pickle
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib

result_dir = 'results'
cache_dir = 'cache'
run_dir_ignore = ['results', 'datasets', 'cache']
IMAGE_NUMBER = 10
SEED_START = 0 # seed是一个全局唯一自层整数，用于生成随机数，也用于作为图片id保存。如果多次跑本脚本，应该手动自增该变量，否则图片命名冲突。

def genImage(model, seed=None):
    # Pick latent vector.
    rnd = np.random.RandomState(seed)
    latents = rnd.randn(1, model.input_shape[1])

    # Generate image.
    fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    images = model.run(latents, None, truncation_psi=0.7, randomize_noise=True, output_transform=fmt)
    return images[0]

def main():
    # Initialize TensorFlow.
    tflib.init_tf()

    # Load pre-trained network.
    url = './models/karras2019stylegan-ffhq-1024x1024.pkl' # karras2019stylegan-ffhq-1024x1024.pkl
    # with dnnlib.util.open_url(url, cache_dir=cache_dir) as f:
        # _G, _D, Gs = pickle.load(f)
        # _G = Instantaneous snapshot of the generator. Mainly useful for resuming a previous training run.
        # _D = Instantaneous snapshot of the discriminator. Mainly useful for resuming a previous training run.
        # Gs = Long-term average of the generator. Yields higher-quality results than the instantaneous snapshot.
    
    _G, _D, Gs = pickle.load(open(url, 'rb'))

    os.makedirs(result_dir, exist_ok=True)

    for i in range(IMAGE_NUMBER):
      SEED_START += 1
      image = genImage(Gs, SEED_START)
      png_filename = os.path.join(result_dir, str(SEED_START) + '.png') # 这里用SEED_START作为图片唯一id。保证在使用时服务不会返回相同图片
      # Save image.
      PIL.Image.fromarray(image, 'RGB').save(png_filename)

if __name__ == "__main__":
    main()